#### Failed bounding boxes model
WARNING: This is put here as example only, didn't use this approach at the end

In [ ]:
#library imports
import os
import random
import math
from datetime import datetime
from collections import Counter
import pandas as pd
import numpy as np

import csv
import cv2
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET

import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

from sklearn.preprocessing import LabelEncoder
import albumentations

#### EDA

In [ ]:
# Looking at data
path_to_csv = r'C:\Users\ismae\Desktop\3rd-year-uni\Statistical-Learning\final_project\data\parte_amistoso_6_35_3.csv'
df_test = pd.read_csv(path_to_csv)
df_test = df_test.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
df_test

In [ ]:
# Checking data types
print(df_test.dtypes)
str(df_test['text'].dtype)

In [ ]:
# Function to aggregate all data into one .csv file

def combine_csv_jpg(folder_path):
    csv_files = []
    jpg_files = []

    # Get all csv and jpg files in the folder
    for file in os.listdir(folder_path):
        if file.endswith(".csv"):
            csv_files.append(file)
        elif file.endswith(".jpg"):
            jpg_files.append(file)

    # Create a dictionary of csv and jpg file names
    file_dict = {}
    for csv_file in csv_files:
        csv_file_name = csv_file.split(".csv")[0]
        for jpg_file in jpg_files:
            jpg_file_name = jpg_file.split(".jpg")[0]
            if csv_file_name == jpg_file_name:
                file_dict[csv_file] = jpg_file

    # Add jpg file name column to csv files
    for csv_file, jpg_file in file_dict.items():
        df = pd.read_csv(os.path.join(folder_path, csv_file))
        df['jpg_file_name'] = jpg_file
        df.to_csv(os.path.join(folder_path, csv_file), index=False)

    # Aggregate all csv files into one file
    all_files = []
    for csv_file in csv_files:
        df = pd.read_csv(os.path.join(folder_path, csv_file))
        all_files.append(df)
    combined_csv = pd.concat(all_files, ignore_index=True)
    combined_csv.to_csv(os.path.join(folder_path, "all_files.csv"), index=False)



In [ ]:
combine_csv_jpg(r"C:\Users\ismae\Desktop\3rd-year-uni\Statistical-Learning\final_project\data")

In [ ]:
path_to_csv = r'C:\Users\ismae\Desktop\3rd-year-uni\Statistical-Learning\final_project\data\all_files.csv'
df = pd.read_csv(path_to_csv, low_memory=False)
df = df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
df

In [ ]:
#Checking new column
print('.unique() of column "jpg_file_name: "', df['jpg_file_name'].unique())
print('\nAmmount of pictures: ', len(df['jpg_file_name'].unique()))

In [ ]:
# Deleting new instances that are not valid in the format
def remove_non_parte_amistoso_rows(df):
    # Create a boolean mask of rows that contain the string "parte_amistoso_" in the 'jpg_file_name' column
    mask = df['jpg_file_name'].str.contains('parte_amistoso_')

    # Filter the DataFrame using the mask
    filtered_df = df[mask]

    # Return the filtered DataFrame
    return filtered_df

df = remove_non_parte_amistoso_rows(df)

In [ ]:
def delete_rows_with_texto_maquina(dataframe):
    # create a boolean mask where campo column contains 'texto maquina'
    mask = dataframe['campo'].str.contains('texto maquina')
    
    # filter out the rows that match the mask
    dataframe = dataframe[~mask]
    
    # return the filtered dataframe
    return dataframe

# apply the function to delete rows containing 'texto maquina'
df_filtered = delete_rows_with_texto_maquina(df)
print('Count of texto maquinas: ', len(df_filtered[df_filtered['campo'] == 'texto maquina']))
df_filtered

In [ ]:
df_filtered.to_csv(r'C:\Users\ismae\Desktop\3rd-year-uni\Statistical-Learning\final_project\before_cleaned_data.csv')

In [ ]:
df_encoded = df_filtered
# create a LabelEncoder object
le = LabelEncoder()

# fit and transform the 'campo' column
df_encoded['campo'] = le.fit_transform(df_encoded['campo'])

# print the encoding used for each label
classes = list(le.classes_)
encoding = list(le.transform(le.classes_))
print(classes)
print(encoding)

In [ ]:
df_classes = pd.DataFrame({'classes': classes, 'encoding': encoding})
df_classes

In [ ]:
df_encoded

In [ ]:
df_encoded['w'].describe()

In [ ]:
df_encoded.to_csv(r'C:\Users\ismae\Desktop\3rd-year-uni\Statistical-Learning\final_project\cleaned_data.csv')
df_classes.to_csv(r'C:\Users\ismae\Desktop\3rd-year-uni\Statistical-Learning\final_project\classes.csv')

#### Adjusting dataset to Transformers format

In [ ]:
def create_dataset(df):
    grouped = df.groupby('jpg_file_name')
    dataset = []
    for i, (filename, group) in enumerate(grouped):
        print('Loading: ', filename)
        image = Image.open('data/' + filename)
        width, height = image.size
        objects = {'id': [], 'area': [], 'bbox': [], 'category': []}
        for j, (_, row) in enumerate(group.iterrows()):
            object_info = {}
            object_info['id'] = row['campo'] + j
            object_info['area'] = int(row['w']) * int(row['h'])  # convert to int before multiplying
            object_info['bbox'] = [int(row['x']), int(row['y']), int(row['w']) if int(row['w']) > 0 else 1, int(row['h']) if int(row['h']) > 0 else 1]
            object_info['category'] = row['campo']
            objects['id'].append(object_info['id'])
            objects['area'].append(object_info['area'])
            objects['bbox'].append(object_info['bbox'])
            objects['category'].append(object_info['category'])
        dataset.append({'image_id': i, 'image': image, 'width': width, 'height': height, 'objects': objects})
    return dataset



In [ ]:
df_transformers = create_dataset(df_encoded)

##### New format
- image_id: the example image id
- image: a PIL.Image.Image object containing the image
- width: width of the image
- height: height of the image
- objects: a dictionary containing bounding box metadata for the objects in the image:
    - id: the annotation id
    - area: the area of the bounding box
    - bbox: the object’s bounding box (in the COCO format )
    - category: the object’s category, with possible all values

In [ ]:
# Check dataset
df_transformers[0]

In [ ]:
for o in df_transformers[0]['objects'].keys():
    print('Length of', o,':', len(df_transformers[0]['objects'][o]))

In [ ]:
train_data = df_transformers[300:500]
test_data = df_transformers[2220:]
train_data

In [ ]:
# Change bbox to ints
for entry in df_transformers:
    entry['objects']['bbox'] = [[int(val) for val in bbox] for bbox in entry['objects']['bbox']]

In [ ]:
from datasets import DatasetDict, Dataset

# Create a Dataset object for each split
train_dataset = Dataset.from_list(train_data)
test_dataset = Dataset.from_list(test_data)

my_dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
})

print(my_dataset)

In [ ]:
my_dataset['train'][0]

In [ ]:
from transformers import AutoImageProcessor

checkpoint = "facebook/detr-resnet-50"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

In [ ]:
transform = albumentations.Compose(
    [
        albumentations.Resize(640, 640),
        albumentations.HorizontalFlip(p=1.0),
        albumentations.RandomBrightnessContrast(p=1.0),
    ],
    bbox_params=albumentations.BboxParams(format="coco", label_fields=["category"]),
)

In [ ]:
def formatted_anns(image_id, category, area, bbox):
    annotations = []
    for i in range(0, len(category)):
        new_ann = {
            "image_id": image_id,
            "category_id": category[i],
            "isCrowd": 0,
            "area": area[i],
            "bbox": list(bbox[i]),
        }
        annotations.append(new_ann)

    return annotations

def transform_aug_ann(examples):
    image_ids = examples["image_id"]
    images, bboxes, area, categories = [], [], [], []
    for image, objects in zip(examples["image"], examples["objects"]):
        image = np.array(image.convert("RGB"))[:, :, ::-1]
        out = transform(image=image, bboxes=objects["bbox"], category=objects["category"])

        area.append(objects["area"])
        images.append(out["image"])
        bboxes.append(out["bboxes"])
        categories.append(out["category"])

    targets = [
        {"image_id": id_, "annotations": formatted_anns(id_, cat_, ar_, box_)}
        for id_, cat_, ar_, box_ in zip(image_ids, categories, area, bboxes)
    ]

    return image_processor(images=images, annotations=targets, return_tensors="pt")

In [ ]:
my_dataset['train'] = my_dataset['train'].with_transform(transform_aug_ann)
my_dataset['train'][16]

In [ ]:
def collate_fn(batch):
    pixel_values = [item["pixel_values"] for item in batch]
    encoding = image_processor.pad_and_create_pixel_mask(pixel_values, return_tensors="pt")
    labels = [item["labels"] for item in batch]
    batch = {}
    batch["pixel_values"] = encoding["pixel_values"]
    batch["pixel_mask"] = encoding["pixel_mask"]
    batch["labels"] = labels
    return batch

#### Training Dert Model

In [ ]:
categories = list(le.classes_)

id2label = {index: x for index, x in enumerate(categories, start=0)}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
from transformers import AutoModelForObjectDetection

model = AutoModelForObjectDetection.from_pretrained(
    checkpoint,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="detr-resnet-50_finetuned_OCR",
    per_device_train_batch_size=8,
    num_train_epochs=10,
    save_steps=200,
    logging_steps=50,
    learning_rate=1e-5,
    weight_decay=1e-4,
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    train_dataset=my_dataset["train"],
    tokenizer=image_processor,
)

trainer.train()

In [ ]:
trainer.push_to_hub()

#### Evaluation

In [ ]:
import json


# format annotations the same as for training, no need for data augmentation
def val_formatted_anns(image_id, objects):
    annotations = []
    for i in range(0, len(objects["id"])):
        new_ann = {
            "id": objects["id"][i],
            "category_id": objects["category"][i],
            "iscrowd": 0,
            "image_id": image_id,
            "area": objects["area"][i],
            "bbox": objects["bbox"][i],
        }
        annotations.append(new_ann)

    return annotations


# Save images and annotations into the files torchvision.datasets.CocoDetection expects
def save_OCR_annotation_file_images(cppe5):
    output_json = {}
    path_output_OCR = f"{os.getcwd()}/OCR/"

    if not os.path.exists(path_output_OCR):
        os.makedirs(path_output_OCR)

    path_anno = os.path.join(path_output_OCR, "OCR_ann.json")
    categories_json = [{"supercategory": "none", "id": id, "name": id2label[id]} for id in id2label]
    output_json["images"] = []
    output_json["annotations"] = []
    for example in cppe5:
        ann = val_formatted_anns(example["image_id"], example["objects"])
        output_json["images"].append(
            {
                "id": example["image_id"],
                "width": example["image"].width,
                "height": example["image"].height,
                "file_name": f"{example['image_id']}.png",
            }
        )
        output_json["annotations"].extend(ann)
    output_json["categories"] = categories_json

    with open(path_anno, "w") as file:
        json.dump(output_json, file, ensure_ascii=False, indent=4)

    for im, img_id in zip(cppe5["image"], cppe5["image_id"]):
        path_img = os.path.join(path_output_OCR, f"{img_id}.png")
        im.save(path_img)

    return path_output_OCR, path_anno

In [ ]:
import torchvision


class CocoDetection(torchvision.datasets.CocoDetection):
    def __init__(self, img_folder, feature_extractor, ann_file):
        super().__init__(img_folder, ann_file)
        self.feature_extractor = feature_extractor

    def __getitem__(self, idx):
        # read in PIL image and target in COCO format
        img, target = super(CocoDetection, self).__getitem__(idx)

        # preprocess image and target: converting target to DETR format,
        # resizing + normalization of both image and target)
        image_id = self.ids[idx]
        target = {"image_id": image_id, "annotations": target}
        encoding = self.feature_extractor(images=img, annotations=target, return_tensors="pt")
        pixel_values = encoding["pixel_values"].squeeze()  # remove batch dimension
        target = encoding["labels"][0]  # remove batch dimension

        return {"pixel_values": pixel_values, "labels": target}


im_processor = AutoImageProcessor.from_pretrained("ismadoukkali/detr-resnet-50_finetuned_OCR")

path_output_OCR, path_anno = save_OCR_annotation_file_images(my_dataset["test"])
test_ds_coco_format = CocoDetection(path_output_OCR, im_processor, path_anno)

In [ ]:
import evaluate
from tqdm import tqdm

model = AutoModelForObjectDetection.from_pretrained("ismadoukkali/detr-resnet-50_finetuned_OCR")
module = evaluate.load("ybelkada/cocoevaluate", coco=test_ds_coco_format.coco)
val_dataloader = torch.utils.data.DataLoader(
    test_ds_coco_format, batch_size=8, shuffle=False, num_workers=0, collate_fn=collate_fn
)

with torch.no_grad():
    for idx, batch in enumerate(tqdm(val_dataloader)):
        pixel_values = batch["pixel_values"]
        pixel_mask = batch["pixel_mask"]

        labels = [
            {k: v for k, v in t.items()} for t in batch["labels"]
        ]  # these are in DETR format, resized + normalized

        # forward pass
        outputs = model(pixel_values=pixel_values, pixel_mask=pixel_mask)

        orig_target_sizes = torch.stack([target["orig_size"] for target in labels], dim=0)
        results = im_processor.post_process(outputs, orig_target_sizes)  # convert outputs of model to COCO api

        module.add(prediction=results, reference=labels)
        del batch

results = module.compute()
print(results)

In [ ]:
from transformers import pipeline
import requests

filename = 'parte_amistoso_1_20_3.jpg'
image = Image.open('data/' + filename)

obj_detector = pipeline(model="ismadoukkali/detr-resnet-50_finetuned_OCR")
obj_detector(image)

In [ ]:
image_processor = AutoImageProcessor.from_pretrained("ismadoukkali/detr-resnet-50_finetuned_OCR")
model = AutoModelForObjectDetection.from_pretrained("ismadoukkali/detr-resnet-50_finetuned_OCR")

with torch.no_grad():
    inputs = image_processor(images=image, return_tensors="pt")
    outputs = model(**inputs)
    target_sizes = torch.tensor([image.size[::-1]])
    results = image_processor.post_process_object_detection(outputs, threshold=0.5, target_sizes=target_sizes)[0]

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
        f"Detected {model.config.id2label[label.item()]} with confidence "
        f"{round(score.item(), 3)} at location {box}"
    )

In [ ]:
from PIL import Image, ImageDraw
draw = ImageDraw.Draw(image)
print(results)
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    print(box)
    box = [round(i, 2) for i in box.tolist()]
    x, y, x2, y2 = tuple(box)
    draw.rectangle((x, y, x2, y2), outline="red", width=1)
    draw.text((x, y), model.config.id2label[label.item()], fill="white")


#### Checking bounding boxes

In [ ]:
from PIL import Image
import matplotlib.patches as patches
# Load the image
img_path = r'C:\Users\ismae\Desktop\3rd-year-uni\Statistical-Learning\final_project\data\parte_amistoso_6_35_3.jpg'
img = Image.open(img_path)

# Create a figure and axes
fig, ax = plt.subplots(figsize=(35, 35))

# Display the image on the axes
ax.imshow(img)

# Loop through the DataFrame and plot the bounding boxes on the image
for i, row in df_test.iterrows():
    if row['campo'] == 'texto maquina':
        continue
    x, y, w, h = row['x'], row['y'], row['w'], row['h']
    object_name = row['campo']
    rect = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor='r', facecolor='none')
    ax.add_patch(rect)
    ax.text(x, y-10, object_name, color='r', fontsize=12)

plt.show()